In [12]:
import moviepy.editor as mp
import pydub
from pydub import AudioSegment
import os

### Separating out audio files from video for later selective muting

In [63]:
stimuli_path = "/Users/f004p74/Documents/dartmouth/projects/fSEND/stimuli/narrated-thought/"
dirlist = os.listdir(stimuli_path)
vidlist = [x for x in dirlist if ('.mp4') in x]

In [64]:
for a,b in enumerate(vidlist):
    b = b.split(".")
    vidlist[a] = b[0]
vidlist

['stutter', 'physical']

In [65]:
for i in vidlist:
    video = mp.VideoFileClip(stimuli_path+str(i)+".mp4")
    video.audio.write_audiofile(stimuli_path+str(i)+".mp3")

MoviePy - Writing audio in /Users/f004p74/Documents/dartmouth/projects/fSEND/stimuli/narrated-thought/stutter.mp3


MoviePy - Done.
MoviePy - Writing audio in /Users/f004p74/Documents/dartmouth/projects/fSEND/stimuli/narrated-thought/physical.mp3


MoviePy - Done.


### Removing selections of audio from the Narrated Thought videos
* Version 1 of stutter: muting the (potential) negatives
* Version 2 of stutter: muting the (potential) positives
* Version 1 of Physical:
* Version 2 of Physical:

In [105]:
stimuli_path = "/Users/f004p74/Documents/dartmouth/projects/fSEND/stimuli/narrated-thought/"
dirlist = os.listdir(stimuli_path)
audiolist = [x for x in dirlist if ('.mp3') in x]
audio = AudioSegment.from_mp3(stimuli_path+audiolist[0])

In [102]:
def converter (minute, second):
    minute_conv = minute * 60000
    second_conv = second * 1000
    return (minute_conv+second_conv)

In [113]:
v1start = [164000,344000,425000,453000,546000,661000]
v1stop = [175000,363000,431000,467000,560000,696000]
v2start = [28000,177000,386000,442000,508000,600000]
v2stop = [31000,197000,406000,443000,514000,660000]
base_start = [0,176000,364000,432000,468000,561000]
base_stop = [163000,343000,424000,452000,545000,660000]

v1_dict = dict(zip(v1start,v1stop))
v2_dict = dict(zip(v2start,v2stop))
base_dict = dict(zip(base_start,base_stop))

In [114]:
for start, stop in base_dict.items():
    segment = audio[start:stop]
    segment.export(stimuli_path+"stutter_base_"+str(start)+"_"+str(stop)+".mp3", format="mp3")

In [115]:
segment = audio[697000:]
segment.export(stimuli_path+"stutter_base_697000_end.mp3", format="mp3")


<_io.BufferedRandom name='/Users/f004p74/Documents/dartmouth/projects/fSEND/stimuli/narrated-thought/stutter_base_697000_end.mp3'>

In [112]:
for start, stop in v1_dict.items():
    replace = AudioSegment.from_mp3(stimuli_path+"removed/stutter_1_"+str(start)+"_"+str(stop)+".mp3")
    newfile = audio[start:stop] + replace

### Stitching together the files to create a reconstructed audio 

In [132]:
v1path = '/Users/f004p74/Documents/dartmouth/projects/fSEND/stimuli/narrated-thought/removed/v1/'
v1_segments = os.listdir(v1path)  
v1_segments.sort()

In [145]:
combined = AudioSegment.empty()

for file in v1_segments:
    combined += AudioSegment.from_mp3(v1path+file)

combined.export(stimuli_path+"stutter_v1.mp3", format="mp3")